In [ ]:
# Initialize duckdb
import duckdb
import pandas as pd

parquet_file = 'train.parquet'

# Start a DuckDB session
con = duckdb.connect(database='my_duckdb.db', read_only=False)

# Create a virtual table that points to the parquet file
con.execute(f"CREATE VIEW IF NOT EXISTS train AS SELECT * FROM parquet_scan('{parquet_file}')")

# Define your query
query = 'SELECT * FROM train WHERE binds = 1 LIMIT 10'

# Execute the query and load the result into a pandas DataFrame
df = pd.read_sql_query(query, con)